In [4]:
from sqlmodel import create_engine, Session, SQLModel, Field, select
from sqltypes import CustomTypeMeta
from sqlalchemy.types import TypeDecorator, String
from typing import Literal, TypeVar
from pydantic import RootModel
S = TypeVar('S', bound=str)

In [5]:
class ValidatedLiteral(type):
  def __new__(cls, LiteralType: type[S]) -> type[TypeDecorator[S]]:
    Type = RootModel[LiteralType]
    def dump(x: S) -> str:
      Type.model_validate(x)
      return x
    def parse(x: str) -> S:
      Type.model_validate(x)
      return x # type: ignore
    return CustomTypeMeta(LiteralType.__name__, (), {}, String, dump=dump, parse=parse)

In [ ]:
class MyLit(SQLModel, table=True):
  id: int | None = Field(default=None, primary_key=True)
  lit: Literal['a', 'b'] = Field(sa_type=ValidatedLiteral())

In [3]:
engine = create_engine('sqlite:///db.sqlite')